### Tilt-series Alignment (IMOD)

#### Shifts between patches in A

Taken from .xf file

In [ ]:
align_mdout_yaml_file = f"{proj_name}_align_mdout.yaml"

if os.path.isfile(align_mdout_yaml_file) is True:
    with open(align_mdout_yaml_file, "r") as f:
        align_md = yaml.load(f.read(), Loader=yaml.FullLoader)

    xf_files = {}
    for ts in list(align_md["stack_output"].keys()):
        fn = align_md["stack_output"][ts]
        xf_files[ts] = fn.replace(".st", ".xf")

    eucs = {}
    for ts in list(xf_files.keys()):
        with open(xf_files[ts], "r") as f:
            lines = f.readlines()
            euc = [
                (float(l.split()[4]) ** 2 + float(l.split()[5]) ** 2) ** 0.5
                for l in lines
            ]
            eucs[ts] = {}
            eucs[ts]["Tilt series"] = align_md["ts"][ts]
            eucs[ts]["Mean shift (A)"] = np.mean(euc)
            eucs[ts]["Shift s.d. (A)"] = np.std(euc)

    eucs_df = pd.DataFrame.from_dict(eucs, orient="index")

    eucs_df

else:
    print(f"{align_mdout_yaml_file} file not found. Was IMOD align run?")

In [ ]:
if os.path.isfile(align_mdout_yaml_file) is True:
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(
        data=eucs_df, x="Tilt series", y="Mean shift (A)", color="cornflowerblue"
    )

    ax.errorbar(
        x=eucs_df.index,
        y=eucs_df["Mean shift (A)"],
        yerr=eucs_df["Shift s.d. (A)"],
        fmt="none",
        c="k",
    )

    plt.gca().set_ylim(bottom=0)

#### Errors in alignment

Taken from taLocals

In [ ]:
stats_yaml_file = f"{proj_name}_imod_align_stats.yaml"

if os.path.isfile(stats_yaml_file) is False:
    print(f"The IMOD stats file {stats_yaml_file} was not found")

else:
    with open(stats_yaml_file, "r") as f:
        stats = yaml.load(f.read(), Loader=yaml.FullLoader)

    stats_df = pd.DataFrame.from_dict(stats).drop(columns=["index"])
    stats_df["Tilt series"] = stats_df["Tilt series"].astype(int, copy=False)
    # pd.set_option("display.max_rows", None)
    display(stats_df)

In [ ]:
if os.path.isfile(stats_yaml_file) is True:
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(
        data=stats_df, x="Tilt series", y="Error mean (nm)", color="cornflowerblue"
    )

    ax.errorbar(
        x=stats_df.index,
        y=stats_df["Error mean (nm)"],
        yerr=stats_df["Error SD (nm)"],
        fmt="none",
        c="k",
    )

    plt.gca().set_ylim(bottom=0)